### Criar um classificador de sentimento aplicando técnicas de PLN
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).
Separar a implementação do seu modelo campeão e a parte de validação do resultado conforme template disponibilizado, a ideia é simular a implementação de um modelo produtivo.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamnentos, varierade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste do seu modelo e na validação que o professor processar (Métrica F1 Score).


[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

In [1]:
#CARREGANDO O DATA FRAME
import pandas as pd
df_full = pd.read_csv('https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv', encoding='utf-8')

df = df_full.sample(n=1000, random_state=42)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 15538 to 21350
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codigo      1000 non-null   int64 
 1   texto       1000 non-null   object
 2   sentimento  1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


In [3]:
# Separando os Data Frames entre treino e teste
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(
    df.copy(), 
    test_size = 0.2, 
    random_state = 42
)

print('Treino: df_train', df_train.shape)
print('Treino: df_test', df_test.shape)

Treino: df_train (800, 3)
Treino: df_test (200, 3)


In [4]:
# Import e downlaod de bibliotecas utilizadas a seguir
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem.rslp import RSLPStemmer
from nltk.stem import SnowballStemmer

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('wordnet')

import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Bom desenvolvimento!

In [5]:
resultado = [] 

In [6]:
stopwords = nltk.corpus.stopwords.words('portuguese')

## Stemming

In [7]:
# Criação das coluna de tokens 
df_train['tokens'] = df_train.texto.apply(word_tokenize)
df_test['tokens'] = df_test.texto.apply(word_tokenize)

In [8]:
# Criação das coluna RSLP Stemmer
rslp = RSLPStemmer()

def stem_pandas(line):
  return ' '.join([rslp.stem(token) for token in line])

df_train['stemmer'] = df_train.tokens.apply(stem_pandas)
df_test['stemmer'] = df_test.tokens.apply(stem_pandas)

In [9]:
# Criação das coluna Snowball Stemmer
snowball = SnowballStemmer("portuguese")

def stemsnow_pandas(line):
  return ' '.join([snowball.stem(token) for token in line])

df_train['stemmersnow'] = df_train.tokens.apply(stemsnow_pandas)
df_test['stemmersnow'] = df_test.tokens.apply(stemsnow_pandas)

## Lemmatização

In [10]:
#!pip install --upgrade spacy
#!python -m spacy download pt_core_news_sm

In [11]:
nlp = spacy.load('pt_core_news_sm')
stopwords_spacy = nlp.Defaults.stop_words

def lemmatizer_spacy(line):
  return ' '.join([token.lemma_ for token in nlp(line)])

df_train['lemma'] = df_train.texto.apply(lemmatizer_spacy)
df_test['lemma'] = df_test.texto.apply(lemmatizer_spacy)

## Vetorização

### Texto

#### CountVectorizer

In [12]:
# vetorização da base de treino
text_count1Vect = CountVectorizer(ngram_range=(1,1))
text_count1Vect.fit(df_train.texto)
text_count1Vect_train = text_count1Vect.transform(df_train.texto)

In [13]:
# vetorização da base de treino
text_count2Vect = CountVectorizer(ngram_range=(2,2))
text_count2Vect.fit(df_train.texto)
text_count2Vect_train = text_count2Vect.transform(df_train.texto)

In [14]:
# vetorização da base de treino
text_count3Vect = CountVectorizer(ngram_range=(3,3))
text_count3Vect.fit(df_train.texto)
text_count3Vect_train = text_count3Vect.transform(df_train.texto)

#### CountVectorizer + Stopwords

In [15]:
# vetorização da base de treino
text_count1StopVect = CountVectorizer(ngram_range=(1,1), stop_words=stopwords)
text_count1StopVect.fit(df_train.texto)
text_count1StopVect_train = text_count1StopVect.transform(df_train.texto)

In [16]:
# vetorização da base de treino
text_count2StopVect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords)
text_count2StopVect.fit(df_train.texto)
text_count2StopVect_train = text_count2StopVect.transform(df_train.texto)

In [17]:
# vetorização da base de treino
text_count3StopVect = CountVectorizer(ngram_range=(3,3), stop_words=stopwords)
text_count3StopVect.fit(df_train.texto)
text_count3StopVect_train = text_count3StopVect.transform(df_train.texto)

#### TfidfVectorizer

In [18]:
text_tfidf1Vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
text_tfidf1Vect.fit(df_train.texto)
text_tfidf1Vect_train = text_tfidf1Vect.transform(df_train.texto)

In [19]:
text_tfidf2Vect = TfidfVectorizer(ngram_range=(2,2), use_idf=True)
text_tfidf2Vect.fit(df_train.texto)
text_tfidf2Vect_train = text_tfidf2Vect.transform(df_train.texto)

In [20]:
text_tfidf3Vect = TfidfVectorizer(ngram_range=(3,3), use_idf=True)
text_tfidf3Vect.fit(df_train.texto)
text_tfidf3Vect_train = text_tfidf3Vect.transform(df_train.texto)

#### TfidfVectorizer + Stopwords

In [21]:
text_tfidf1StopVect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stopwords)
text_tfidf1StopVect.fit(df_train.texto)
text_tfidf1StopVect_train = text_tfidf1StopVect.transform(df_train.texto)

In [22]:
text_tfidf2StopVect = TfidfVectorizer(ngram_range=(2,2), use_idf=True, stop_words=stopwords)
text_tfidf2StopVect.fit(df_train.texto)
text_tfidf2StopVect_train = text_tfidf2StopVect.transform(df_train.texto)

In [23]:
text_tfidf3StopVect = TfidfVectorizer(ngram_range=(3,3), use_idf=True, stop_words=stopwords)
text_tfidf3StopVect.fit(df_train.texto)
text_tfidf3StopVect_train = text_tfidf3StopVect.transform(df_train.texto)

### RSLP Stemmer

#### CountVectorizer

In [24]:
# vetorização da base de treino
rslp_count1Vect = CountVectorizer(ngram_range=(1,1))
rslp_count1Vect.fit(df_train.stemmer)
rslp_count1Vect_train = rslp_count1Vect.transform(df_train.stemmer)

In [25]:
# vetorização da base de treino
rslp_count2Vect = CountVectorizer(ngram_range=(2,2))
rslp_count2Vect.fit(df_train.stemmer)
rslp_count2Vect_train = rslp_count2Vect.transform(df_train.stemmer)

In [26]:
# vetorização da base de treino
rslp_count3Vect = CountVectorizer(ngram_range=(3,3))
rslp_count3Vect.fit(df_train.stemmer)
rslp_count3Vect_train = rslp_count3Vect.transform(df_train.stemmer)

#### CountVectorizer + Stopwords

In [27]:
# vetorização da base de treino
rslp_count1StopVect = CountVectorizer(ngram_range=(1,1), stop_words=stopwords)
rslp_count1StopVect.fit(df_train.stemmer)
rslp_count1StopVect_train = rslp_count1StopVect.transform(df_train.stemmer)

In [28]:
# vetorização da base de treino
rslp_count2StopVect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords)
rslp_count2StopVect.fit(df_train.stemmer)
rslp_count2StopVect_train = rslp_count2StopVect.transform(df_train.stemmer)

In [29]:
# vetorização da base de treino
rslp_count3StopVect = CountVectorizer(ngram_range=(3,3), stop_words=stopwords)
rslp_count3StopVect.fit(df_train.stemmer)
rslp_count3StopVect_train = rslp_count3StopVect.transform(df_train.stemmer)

#### TfidfVectorizer

In [30]:
rslp_tfidf1Vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
rslp_tfidf1Vect.fit(df_train.stemmer)
rslp_tfidf1Vect_train = rslp_tfidf1Vect.transform(df_train.stemmer)

In [31]:
rslp_tfidf2Vect = TfidfVectorizer(ngram_range=(2,2), use_idf=True)
rslp_tfidf2Vect.fit(df_train.stemmer)
rslp_tfidf2Vect_train = rslp_tfidf2Vect.transform(df_train.stemmer)

In [32]:
rslp_tfidf3Vect = TfidfVectorizer(ngram_range=(3,3), use_idf=True)
rslp_tfidf3Vect.fit(df_train.stemmer)
rslp_tfidf3Vect_train = rslp_tfidf3Vect.transform(df_train.stemmer)

#### TfidfVectorizer + Stopwords

In [33]:
rslp_tfidf1StopVect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stopwords)
rslp_tfidf1StopVect.fit(df_train.stemmer)
rslp_tfidf1StopVect_train = rslp_tfidf1StopVect.transform(df_train.stemmer)

In [34]:
rslp_tfidf2StopVect = TfidfVectorizer(ngram_range=(2,2), use_idf=True, stop_words=stopwords)
rslp_tfidf2StopVect.fit(df_train.stemmer)
rslp_tfidf2StopVect_train = rslp_tfidf2StopVect.transform(df_train.stemmer)

In [35]:
rslp_tfidf3StopVect = TfidfVectorizer(ngram_range=(3,3), use_idf=True, stop_words=stopwords)
rslp_tfidf3StopVect.fit(df_train.stemmer)
rslp_tfidf3StopVect_train = rslp_tfidf3StopVect.transform(df_train.stemmer)

### Snowball Stemmer

#### CountVectorizer

In [36]:
# vetorização da base de treino
snow_count1Vect = CountVectorizer(ngram_range=(1,1))
snow_count1Vect.fit(df_train.stemmersnow)
snow_count1Vect_train = snow_count1Vect.transform(df_train.stemmersnow)

In [37]:
# vetorização da base de treino
snow_count2Vect = CountVectorizer(ngram_range=(2,2))
snow_count2Vect.fit(df_train.stemmersnow)
snow_count2Vect_train = snow_count2Vect.transform(df_train.stemmersnow)

In [38]:
# vetorização da base de treino
snow_count3Vect = CountVectorizer(ngram_range=(3,3))
snow_count3Vect.fit(df_train.stemmersnow)
snow_count3Vect_train = snow_count3Vect.transform(df_train.stemmersnow)

#### CountVectorizer + Stopwords

In [39]:
# vetorização da base de treino
snow_count1StopVect = CountVectorizer(ngram_range=(1,1), stop_words=stopwords)
snow_count1StopVect.fit(df_train.stemmersnow)
snow_count1StopVect_train = snow_count1StopVect.transform(df_train.stemmersnow)

In [40]:
# vetorização da base de treino
snow_count2StopVect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords)
snow_count2StopVect.fit(df_train.stemmersnow)
snow_count2StopVect_train = snow_count2StopVect.transform(df_train.stemmersnow)

In [41]:
# vetorização da base de treino
snow_count3StopVect = CountVectorizer(ngram_range=(3,3), stop_words=stopwords)
snow_count3StopVect.fit(df_train.stemmersnow)
snow_count3StopVect_train = snow_count3StopVect.transform(df_train.stemmersnow)

#### TfidfVectorizer

In [42]:
snow_tfidf1Vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
snow_tfidf1Vect.fit(df_train.stemmersnow)
snow_tfidf1Vect_train = snow_tfidf1Vect.transform(df_train.stemmersnow)

In [43]:
snow_tfidf2Vect = TfidfVectorizer(ngram_range=(2,2), use_idf=True)
snow_tfidf2Vect.fit(df_train.stemmersnow)
snow_tfidf2Vect_train = snow_tfidf2Vect.transform(df_train.stemmersnow)

In [44]:
snow_tfidf3Vect = TfidfVectorizer(ngram_range=(3,3), use_idf=True)
snow_tfidf3Vect.fit(df_train.stemmersnow)
snow_tfidf3Vect_train = snow_tfidf3Vect.transform(df_train.stemmersnow)

#### TfidfVectorizer + Stopwords

In [45]:
snow_tfidf1StopVect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stopwords)
snow_tfidf1StopVect.fit(df_train.stemmersnow)
snow_tfidf1StopVect_train = snow_tfidf1StopVect.transform(df_train.stemmersnow)

In [46]:
snow_tfidf2StopVect = TfidfVectorizer(ngram_range=(2,2), use_idf=True, stop_words=stopwords)
snow_tfidf2StopVect.fit(df_train.stemmersnow)
snow_tfidf2StopVect_train = snow_tfidf2StopVect.transform(df_train.stemmersnow)

In [47]:
snow_tfidf3StopVect = TfidfVectorizer(ngram_range=(3,3), use_idf=True, stop_words=stopwords)
snow_tfidf3StopVect.fit(df_train.stemmersnow)
snow_tfidf3StopVect_train = snow_tfidf3StopVect.transform(df_train.stemmersnow)

### Lemma

#### CountVectorizer

In [48]:
# vetorização da base de treino
lemma_count1Vect = CountVectorizer(ngram_range=(1,1))
lemma_count1Vect.fit(df_train.lemma)
lemma_count1Vect_train = lemma_count1Vect.transform(df_train.lemma)

In [49]:
# vetorização da base de treino
lemma_count2Vect = CountVectorizer(ngram_range=(2,2))
lemma_count2Vect.fit(df_train.lemma)
lemma_count2Vect_train = lemma_count2Vect.transform(df_train.lemma)

In [50]:
# vetorização da base de treino
lemma_count3Vect = CountVectorizer(ngram_range=(3,3))
lemma_count3Vect.fit(df_train.lemma)
lemma_count3Vect_train = lemma_count3Vect.transform(df_train.lemma)

#### CountVectorizer + Stopwords

In [51]:
# vetorização da base de treino
lemma_count1StopVect = CountVectorizer(ngram_range=(1,1), stop_words=stopwords_spacy)
lemma_count1StopVect.fit(df_train.lemma)
lemma_count1StopVect_train = lemma_count1StopVect.transform(df_train.lemma)

In [52]:
# vetorização da base de treino
lemma_count2StopVect = CountVectorizer(ngram_range=(2,2), stop_words=stopwords_spacy)
lemma_count2StopVect.fit(df_train.lemma)
lemma_count2StopVect_train = lemma_count2StopVect.transform(df_train.lemma)

In [53]:
# vetorização da base de treino
lemma_count3StopVect = CountVectorizer(ngram_range=(3,3), stop_words=stopwords_spacy)
lemma_count3StopVect.fit(df_train.lemma)
lemma_count3StopVect_train = lemma_count3StopVect.transform(df_train.lemma)

#### TfidfVectorizer

In [54]:
lemma_tfidf1Vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
lemma_tfidf1Vect.fit(df_train.lemma)
lemma_tfidf1Vect_train = lemma_tfidf1Vect.transform(df_train.lemma)

In [55]:
lemma_tfidf2Vect = TfidfVectorizer(ngram_range=(2,2), use_idf=True)
lemma_tfidf2Vect.fit(df_train.lemma)
lemma_tfidf2Vect_train = lemma_tfidf2Vect.transform(df_train.lemma)

In [56]:
lemma_tfidf3Vect = TfidfVectorizer(ngram_range=(3,3), use_idf=True)
lemma_tfidf3Vect.fit(df_train.lemma)
lemma_tfidf3Vect_train = lemma_tfidf3Vect.transform(df_train.lemma)

#### TfidfVectorizer + Stopwords

In [57]:
lemma_tfidf1StopVect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stopwords_spacy)
lemma_tfidf1StopVect.fit(df_train.lemma)
lemma_tfidf1StopVect_train = lemma_tfidf1StopVect.transform(df_train.lemma)

In [58]:
lemma_tfidf2StopVect = TfidfVectorizer(ngram_range=(2,2), use_idf=True, stop_words=stopwords_spacy)
lemma_tfidf2StopVect.fit(df_train.lemma)
lemma_tfidf2StopVect_train = lemma_tfidf2StopVect.transform(df_train.lemma)

In [59]:
lemma_tfidf3StopVect = TfidfVectorizer(ngram_range=(3,3), use_idf=True, stop_words=stopwords_spacy)
lemma_tfidf3StopVect.fit(df_train.lemma)
lemma_tfidf3StopVect_train = lemma_tfidf3StopVect.transform(df_train.lemma)

## Treinamento e escoragem

### Arvore de Decisão

#### Texto



##### CountVectorizer

In [60]:
# Texto CountVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Unigrama com arvore de decisão', score])
print('Texto CountVectorizer Unigrama com arvore de decisão:', score)

Texto CountVectorizer Unigrama com arvore de decisão: 0.6751708342719831


In [61]:
# Texto CountVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Bigrama com arvore de decisão', score])
print('Texto CountVectorizer Bigrama com arvore de decisão:', score)

Texto CountVectorizer Bigrama com arvore de decisão: 0.6460427807486632


In [62]:
# Texto CountVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Trigrama com arvore de decisão', score])
print('Texto CountVectorizer Trigrama com arvore de decisão:', score)

Texto CountVectorizer Trigrama com arvore de decisão: 0.6105035526722274


##### CountVectorizer + Stopwords

In [63]:
# Texto CountVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Unigrama e stopwords com arvore de decisão', score])
print('Texto CountVectorizer Unigrama e stopwords com arvore de decisão:', score)

Texto CountVectorizer Unigrama e stopwords com arvore de decisão: 0.6878427159432746


In [64]:
# Texto CountVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Bigrama e stopwords com arvore de decisão', score])
print('Texto CountVectorizer Bigrama e stopwords com arvore de decisão:', score)

Texto CountVectorizer Bigrama e stopwords com arvore de decisão: 0.6689280197713933


In [65]:
# Texto CountVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Trigrama e stopwords com arvore de decisão', score])
print('Texto CountVectorizer Trigrama e stopwords com arvore de decisão:', score)

Texto CountVectorizer Trigrama e stopwords com arvore de decisão: 0.6578361325868657


##### TfidfVectorizer

In [66]:
# Texto TfidfVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Unigrama com arvore de decisão', score])
print('Texto TfidfVectorizer Unigrama com arvore de decisão:', score)

Texto TfidfVectorizer Unigrama com arvore de decisão: 0.6560111726245649


In [67]:
# Texto TfidfVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Bigrama com arvore de decisão', score])
print('Texto TfidfVectorizer Bigrama com arvore de decisão:', score)

Texto TfidfVectorizer Bigrama com arvore de decisão: 0.655388367729831


In [68]:
# Texto TfidfVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Trigrama com arvore de decisão', score])
print('Texto TfidfVectorizer Trigrama com arvore de decisão:', score)

Texto TfidfVectorizer Trigrama com arvore de decisão: 0.5798850574712644


##### TfidfVectorizer + Stopwords

In [69]:
# Texto TfidfVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Unigrama e stopwords com arvore de decisão', score])
print('Texto TfidfVectorizer Unigrama e stopwords com arvore de decisão:', score)

Texto TfidfVectorizer Unigrama e stopwords com arvore de decisão: 0.6305554999499549


In [70]:
# Texto TfidfVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Bigrama e stopwords com arvore de decisão', score])
print('Texto TfidfVectorizer Bigrama e stopwords com arvore de decisão:', score)

Texto TfidfVectorizer Bigrama e stopwords com arvore de decisão: 0.6302964743589743


In [71]:
# Texto TfidfVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Trigrama e stopwords com arvore de decisão', score])
print('Texto TfidfVectorizer Trigrama e stopwords com arvore de decisão:', score)

Texto TfidfVectorizer Trigrama e stopwords com arvore de decisão: 0.6408363636363638


#### RSLP Stemmer



##### CountVectorizer

In [72]:
# RSLP CountVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Unigrama com arvore de decisão', score])
print('RSLP CountVectorizer Unigrama com arvore de decisão:', score)

RSLP CountVectorizer Unigrama com arvore de decisão: 0.612313133634669


In [73]:
# RSLP CountVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Bigrama com arvore de decisão', score])
print('RSLP CountVectorizer Bigrama com arvore de decisão:', score)

RSLP CountVectorizer Bigrama com arvore de decisão: 0.6241909150757077


In [74]:
# RSLP CountVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Trigrama com arvore de decisão', score])
print('RSLP CountVectorizer Trigrama com arvore de decisão:', score)

RSLP CountVectorizer Trigrama com arvore de decisão: 0.6217312120577768


##### CountVectorizer + Stopwords

In [75]:
# RSLP CountVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Unigrama e stopwords com arvore de decisão', score])
print('RSLP CountVectorizer Unigrama e stopwords com arvore de decisão:', score)

RSLP CountVectorizer Unigrama e stopwords com arvore de decisão: 0.6547619047619048


In [76]:
# RSLP CountVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Bigrama e stopwords com arvore de decisão', score])
print('RSLP CountVectorizer Bigrama e stopwords com arvore de decisão:', score)

RSLP CountVectorizer Bigrama e stopwords com arvore de decisão: 0.6735751442353164


In [77]:
# RSLP CountVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Trigrama e stopwords com arvore de decisão', score])
print('RSLP CountVectorizer Trigrama e stopwords com arvore de decisão:', score)

RSLP CountVectorizer Trigrama e stopwords com arvore de decisão: 0.6677408637873755


##### TfidfVectorizer

In [78]:
# RSLP TfidfVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Unigrama com arvore de decisão', score])
print('RSLP TfidfVectorizer Unigrama com arvore de decisão:', score)

RSLP TfidfVectorizer Unigrama com arvore de decisão: 0.658819639735502


In [79]:
# RSLP TfidfVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Bigrama com arvore de decisão', score])
print('RSLP TfidfVectorizer Bigrama com arvore de decisão:', score)

RSLP TfidfVectorizer Bigrama com arvore de decisão: 0.6475524105048058


In [80]:
# RSLP TfidfVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Trigrama com arvore de decisão', score])
print('RSLP TfidfVectorizer Trigrama com arvore de decisão:', score)

RSLP TfidfVectorizer Trigrama com arvore de decisão: 0.6316574653416759


##### TfidfVectorizer + Stopwords

In [81]:
# RSLP TfidfVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Unigrama e stopwords com arvore de decisão', score])
print('RSLP TfidfVectorizer Unigrama e stopwords com arvore de decisão:', score)

RSLP TfidfVectorizer Unigrama e stopwords com arvore de decisão: 0.6309299483079117


In [82]:
# RSLP TfidfVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Bigrama e stopwords com arvore de decisão', score])
print('RSLP TfidfVectorizer Bigrama e stopwords com arvore de decisão:', score)

RSLP TfidfVectorizer Bigrama e stopwords com arvore de decisão: 0.6735751442353164


In [83]:
# RSLP TfidfVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(rslp_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Trigrama e stopwords com arvore de decisão', score])
print('RSLP TfidfVectorizer Trigrama e stopwords com arvore de decisão:', score)

RSLP TfidfVectorizer Trigrama e stopwords com arvore de decisão: 0.6677408637873755


#### Snowball Stemmer



##### CountVectorizer

In [84]:
# Snowball CountVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Unigrama com arvore de decisão', score])
print('Snowball CountVectorizer Unigrama com arvore de decisão:', score)

Snowball CountVectorizer Unigrama com arvore de decisão: 0.6535


In [85]:
# Snowball CountVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Bigrama com arvore de decisão', score])
print('Snowball CountVectorizer Bigrama com arvore de decisão:', score)

Snowball CountVectorizer Bigrama com arvore de decisão: 0.6033650318452645


In [86]:
# Snowball CountVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Trigrama com arvore de decisão', score])
print('Snowball CountVectorizer Trigrama com arvore de decisão:', score)

Snowball CountVectorizer Trigrama com arvore de decisão: 0.628416877028489


##### CountVectorizer + Stopwords

In [87]:
# Snowball CountVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Unigrama e stopwords com arvore de decisão', score])
print('Snowball CountVectorizer Unigrama e stopwords com arvore de decisão:', score)

Snowball CountVectorizer Unigrama e stopwords com arvore de decisão: 0.6442068965517241


In [88]:
# Snowball CountVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Bigrama e stopwords com arvore de decisão', score])
print('Snowball CountVectorizer Bigrama e stopwords com arvore de decisão:', score)

Snowball CountVectorizer Bigrama e stopwords com arvore de decisão: 0.6754966887417219


In [89]:
# Snowball CountVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Trigrama e stopwords com arvore de decisão', score])
print('Snowball CountVectorizer Trigrama e stopwords com arvore de decisão:', score)

Snowball CountVectorizer Trigrama e stopwords com arvore de decisão: 0.6677408637873755


##### TfidfVectorizer

In [90]:
# Snowball TfidfVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Unigrama com arvore de decisão', score])
print('Snowball TfidfVectorizer Unigrama com arvore de decisão:', score)

Snowball TfidfVectorizer Unigrama com arvore de decisão: 0.6940265343544385


In [91]:
# Snowball TfidfVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Bigrama com arvore de decisão', score])
print('Snowball TfidfVectorizer Bigrama com arvore de decisão:', score)

Snowball TfidfVectorizer Bigrama com arvore de decisão: 0.5285712443584785


In [92]:
# Snowball TfidfVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Trigrama com arvore de decisão', score])
print('Snowball TfidfVectorizer Trigrama com arvore de decisão:', score)

Snowball TfidfVectorizer Trigrama com arvore de decisão: 0.612261089073206


##### TfidfVectorizer + Stopwords

In [93]:
# Snowball TfidfVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Unigrama e stopwords com arvore de decisão', score])
print('Snowball TfidfVectorizer Unigrama e stopwords com arvore de decisão:', score)

Snowball TfidfVectorizer Unigrama e stopwords com arvore de decisão: 0.632989898989899


In [94]:
# Snowball TfidfVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Bigrama e stopwords com arvore de decisão', score])
print('Snowball TfidfVectorizer Bigrama e stopwords com arvore de decisão:', score)

Snowball TfidfVectorizer Bigrama e stopwords com arvore de decisão: 0.6668


In [95]:
# Snowball TfidfVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(snow_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Trigrama e stopwords com arvore de decisão', score])
print('Snowball TfidfVectorizer Trigrama e stopwords com arvore de decisão:', score)

Snowball TfidfVectorizer Trigrama e stopwords com arvore de decisão: 0.6677408637873755


#### Lemma



##### CountVectorizer

In [96]:
# Lemma CountVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Unigrama com arvore de decisão', score])
print('Lemma CountVectorizer Unigrama com arvore de decisão:', score)

Lemma CountVectorizer Unigrama com arvore de decisão: 0.6702974204884012


In [97]:
# Lemma CountVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Bigrama com arvore de decisão', score])
print('Lemma CountVectorizer Bigrama com arvore de decisão:', score)

Lemma CountVectorizer Bigrama com arvore de decisão: 0.595648452929559


In [98]:
# Lemma CountVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Trigrama com arvore de decisão', score])
print('Lemma CountVectorizer Trigrama com arvore de decisão:', score)

Lemma CountVectorizer Trigrama com arvore de decisão: 0.6135963994138579


##### CountVectorizer + Stopwords

In [99]:
# Lemma CountVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Unigrama e stopwords com arvore de decisão', score])
print('Lemma CountVectorizer Unigrama e stopwords com arvore de decisão:', score)

Lemma CountVectorizer Unigrama e stopwords com arvore de decisão: 0.6450543780265143


In [100]:
# Lemma CountVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Bigrama e stopwords com arvore de decisão', score])
print('Lemma CountVectorizer Bigrama e stopwords com arvore de decisão:', score)

Lemma CountVectorizer Bigrama e stopwords com arvore de decisão: 0.6577130977130976


In [101]:
# Lemma CountVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Trigrama e stopwords com arvore de decisão', score])
print('Lemma CountVectorizer Trigrama e stopwords com arvore de decisão:', score)

Lemma CountVectorizer Trigrama e stopwords com arvore de decisão: 0.6754966887417219


##### TfidfVectorizer

In [102]:
# Lemma TfidfVectorizer Unigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf1Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Unigrama com arvore de decisão', score])
print('Lemma TfidfVectorizer Unigrama com arvore de decisão:', score)

Lemma TfidfVectorizer Unigrama com arvore de decisão: 0.6488691796008869


In [103]:
# Lemma TfidfVectorizer Bigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf2Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Bigrama com arvore de decisão', score])
print('Lemma TfidfVectorizer Bigrama com arvore de decisão:', score)

Lemma TfidfVectorizer Bigrama com arvore de decisão: 0.635861491787844


In [104]:
# Lemma TfidfVectorizer Trigrama com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf3Vect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Trigrama com arvore de decisão', score])
print('Lemma TfidfVectorizer Trigrama com arvore de decisão:', score)

Lemma TfidfVectorizer Trigrama com arvore de decisão: 0.6298660247016956


##### TfidfVectorizer + Stopwords

In [105]:
# Lemma TfidfVectorizer Unigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf1StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Unigrama e stopwords com arvore de decisão', score])
print('Lemma TfidfVectorizer Unigrama e stopwords com arvore de decisão:', score)

Lemma TfidfVectorizer Unigrama e stopwords com arvore de decisão: 0.6910903426791277


In [106]:
# Lemma TfidfVectorizer Bigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf2StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Bigrama e stopwords com arvore de decisão', score])
print('Lemma TfidfVectorizer Bigrama e stopwords com arvore de decisão:', score)

Lemma TfidfVectorizer Bigrama e stopwords com arvore de decisão: 0.6650730607041289


In [107]:
# Lemma TfidfVectorizer Trigrama e stopwords com arvore de decisão
# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(lemma_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf3StopVect.transform(df_test.texto)
y_prediction = tree.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Trigrama e stopwords com arvore de decisão', score])
print('Lemma TfidfVectorizer Trigrama e stopwords com arvore de decisão:', score)

Lemma TfidfVectorizer Trigrama e stopwords com arvore de decisão: 0.6668


### Naive Bayes

#### Texto



##### CountVectorizer

In [108]:
# Texto CountVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Unigrama com Naive Bayes', score])
print('Texto CountVectorizer Unigrama com Naive Bayes:', score)

Texto CountVectorizer Unigrama com Naive Bayes: 0.7951076492210748


In [109]:
# Texto CountVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Bigrama com Naive Bayes', score])
print('Texto CountVectorizer Bigrama com Naive Bayes:', score)

Texto CountVectorizer Bigrama com Naive Bayes: 0.8001600640256101


In [110]:
# Texto CountVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Trigrama com Naive Bayes', score])
print('Texto CountVectorizer Trigrama com Naive Bayes:', score)

Texto CountVectorizer Trigrama com Naive Bayes: 0.7200840756681012


##### CountVectorizer + Stopwords

In [111]:
# Texto CountVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Unigrama e stopwords com Naive Bayes', score])
print('Texto CountVectorizer Unigrama e stopwords com Naive Bayes:', score)

Texto CountVectorizer Unigrama e stopwords com Naive Bayes: 0.7900630063006301


In [112]:
# Texto CountVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Bigrama e stopwords com Naive Bayes', score])
print('Texto CountVectorizer Bigrama e stopwords com Naive Bayes:', score)

Texto CountVectorizer Bigrama e stopwords com Naive Bayes: 0.744980874521863


In [113]:
# Texto CountVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_count3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto CountVectorizer Trigrama e stopwords com Naive Bayes', score])
print('Texto CountVectorizer Trigrama e stopwords com Naive Bayes:', score)

Texto CountVectorizer Trigrama e stopwords com Naive Bayes: 0.643977024491789


##### TfidfVectorizer

In [114]:
# Texto TfidfVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Unigrama com Naive Bayes', score])
print('Texto TfidfVectorizer Unigrama com Naive Bayes:', score)

Texto TfidfVectorizer Unigrama com Naive Bayes: 0.799395220164451


In [115]:
# Texto TfidfVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Bigrama com Naive Bayes', score])
print('Texto TfidfVectorizer Bigrama com Naive Bayes:', score)

Texto TfidfVectorizer Bigrama com Naive Bayes: 0.7910161186331399


In [116]:
# Texto TfidfVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Trigrama com Naive Bayes', score])
print('Texto TfidfVectorizer Trigrama com Naive Bayes:', score)

Texto TfidfVectorizer Trigrama com Naive Bayes: 0.72230032713636


##### TfidfVectorizer + Stopwords

In [117]:
# Texto TfidfVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Unigrama e stopwords com Naive Bayes', score])
print('Texto TfidfVectorizer Unigrama e stopwords com Naive Bayes:', score)

Texto TfidfVectorizer Unigrama e stopwords com Naive Bayes: 0.7739412484700123


In [118]:
# Texto TfidfVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Bigrama e stopwords com Naive Bayes', score])
print('Texto TfidfVectorizer Bigrama e stopwords com Naive Bayes:', score)

Texto TfidfVectorizer Bigrama e stopwords com Naive Bayes: 0.7431655844155844


In [119]:
# Texto TfidfVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = text_tfidf3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Texto TfidfVectorizer Trigrama e stopwords com Naive Bayes', score])
print('Texto TfidfVectorizer Trigrama e stopwords com Naive Bayes:', score)

Texto TfidfVectorizer Trigrama e stopwords com Naive Bayes: 0.6433066666666666


#### RSLP Stemmer



##### CountVectorizer

In [120]:
# RSLP CountVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Unigrama com Naive Bayes', score])
print('RSLP CountVectorizer Unigrama com Naive Bayes:', score)

RSLP CountVectorizer Unigrama com Naive Bayes: 0.6881387802867097


In [121]:
# RSLP CountVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Bigrama com Naive Bayes', score])
print('RSLP CountVectorizer Bigrama com Naive Bayes:', score)

RSLP CountVectorizer Bigrama com Naive Bayes: 0.6595786041103162


In [122]:
# RSLP CountVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Trigrama com Naive Bayes', score])
print('RSLP CountVectorizer Trigrama com Naive Bayes:', score)

RSLP CountVectorizer Trigrama com Naive Bayes: 0.6218357487922705


##### CountVectorizer + Stopwords

In [123]:
# RSLP CountVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Unigrama e stopwords com Naive Bayes', score])
print('RSLP CountVectorizer Unigrama e stopwords com Naive Bayes:', score)

RSLP CountVectorizer Unigrama e stopwords com Naive Bayes: 0.672140001527845


In [124]:
# RSLP CountVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Bigrama e stopwords com Naive Bayes', score])
print('RSLP CountVectorizer Bigrama e stopwords com Naive Bayes:', score)

RSLP CountVectorizer Bigrama e stopwords com Naive Bayes: 0.6223629786899426


In [125]:
# RSLP CountVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_count3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP CountVectorizer Trigrama e stopwords com Naive Bayes', score])
print('RSLP CountVectorizer Trigrama e stopwords com Naive Bayes:', score)

RSLP CountVectorizer Trigrama e stopwords com Naive Bayes: 0.6650730607041289


##### TfidfVectorizer

In [126]:
# RSLP TfidfVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Unigrama com Naive Bayes', score])
print('RSLP TfidfVectorizer Unigrama com Naive Bayes:', score)

RSLP TfidfVectorizer Unigrama com Naive Bayes: 0.6713932980599647


In [127]:
# RSLP TfidfVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Bigrama com Naive Bayes', score])
print('RSLP TfidfVectorizer Bigrama com Naive Bayes:', score)

RSLP TfidfVectorizer Bigrama com Naive Bayes: 0.6354932301740811


In [128]:
# RSLP TfidfVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Trigrama com Naive Bayes', score])
print('RSLP TfidfVectorizer Trigrama com Naive Bayes:', score)

RSLP TfidfVectorizer Trigrama com Naive Bayes: 0.6007578025087752


##### TfidfVectorizer + Stopwords

In [129]:
# RSLP TfidfVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Unigrama e stopwords com Naive Bayes', score])
print('RSLP TfidfVectorizer Unigrama e stopwords com Naive Bayes:', score)

RSLP TfidfVectorizer Unigrama e stopwords com Naive Bayes: 0.635861491787844


In [130]:
# RSLP TfidfVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Bigrama e stopwords com Naive Bayes', score])
print('RSLP TfidfVectorizer Bigrama e stopwords com Naive Bayes:', score)

RSLP TfidfVectorizer Bigrama e stopwords com Naive Bayes: 0.618843495878549


In [131]:
# RSLP TfidfVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(rslp_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = rslp_tfidf3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['RSLP TfidfVectorizer Trigrama e stopwords com Naive Bayes', score])
print('RSLP TfidfVectorizer Trigrama e stopwords com Naive Bayes:', score)

RSLP TfidfVectorizer Trigrama e stopwords com Naive Bayes: 0.6650730607041289


#### Snowball Stemmer



##### CountVectorizer

In [132]:
# Snowball CountVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Unigrama com Naive Bayes', score])
print('Snowball CountVectorizer Unigrama com Naive Bayes:', score)

Snowball CountVectorizer Unigrama com Naive Bayes: 0.7059505645407385


In [133]:
# Snowball CountVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Bigrama com Naive Bayes', score])
print('Snowball CountVectorizer Bigrama com Naive Bayes:', score)

Snowball CountVectorizer Bigrama com Naive Bayes: 0.6299629962996299


In [134]:
# Snowball CountVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Trigrama com Naive Bayes', score])
print('Snowball CountVectorizer Trigrama com Naive Bayes:', score)

Snowball CountVectorizer Trigrama com Naive Bayes: 0.5701291162045841


##### CountVectorizer + Stopwords

In [135]:
# Snowball CountVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Unigrama e stopwords com Naive Bayes', score])
print('Snowball CountVectorizer Unigrama e stopwords com Naive Bayes:', score)

Snowball CountVectorizer Unigrama e stopwords com Naive Bayes: 0.6605104594134721


In [136]:
# Snowball CountVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Bigrama e stopwords com Naive Bayes', score])
print('Snowball CountVectorizer Bigrama e stopwords com Naive Bayes:', score)

Snowball CountVectorizer Bigrama e stopwords com Naive Bayes: 0.586125


In [137]:
# Snowball CountVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_count3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball CountVectorizer Trigrama e stopwords com Naive Bayes', score])
print('Snowball CountVectorizer Trigrama e stopwords com Naive Bayes:', score)

Snowball CountVectorizer Trigrama e stopwords com Naive Bayes: 0.6650730607041289


##### TfidfVectorizer

In [138]:
# Snowball TfidfVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Unigrama com Naive Bayes', score])
print('Snowball TfidfVectorizer Unigrama com Naive Bayes:', score)

Snowball TfidfVectorizer Unigrama com Naive Bayes: 0.6963192904656319


In [139]:
# Snowball TfidfVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Bigrama com Naive Bayes', score])
print('Snowball TfidfVectorizer Bigrama com Naive Bayes:', score)

Snowball TfidfVectorizer Bigrama com Naive Bayes: 0.6404387155133423


In [140]:
# Snowball TfidfVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Trigrama com Naive Bayes', score])
print('Snowball TfidfVectorizer Trigrama com Naive Bayes:', score)

Snowball TfidfVectorizer Trigrama com Naive Bayes: 0.6164827586206897


##### TfidfVectorizer + Stopwords

In [141]:
# Snowball TfidfVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Unigrama e stopwords com Naive Bayes', score])
print('Snowball TfidfVectorizer Unigrama e stopwords com Naive Bayes:', score)

Snowball TfidfVectorizer Unigrama e stopwords com Naive Bayes: 0.6913846153846154


In [142]:
# Snowball TfidfVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Bigrama e stopwords com Naive Bayes', score])
print('Snowball TfidfVectorizer Bigrama e stopwords com Naive Bayes:', score)

Snowball TfidfVectorizer Bigrama e stopwords com Naive Bayes: 0.605359565807327


In [143]:
# Snowball TfidfVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(snow_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = snow_tfidf3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Snowball TfidfVectorizer Trigrama e stopwords com Naive Bayes', score])
print('Snowball TfidfVectorizer Trigrama e stopwords com Naive Bayes:', score)

Snowball TfidfVectorizer Trigrama e stopwords com Naive Bayes: 0.666045564422663


#### Lemma



##### CountVectorizer

In [144]:
# Lemma CountVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_count1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Unigrama com Naive Bayes', score])
print('Lemma CountVectorizer Unigrama com Naive Bayes:', score)

Lemma CountVectorizer Unigrama com Naive Bayes: 0.73008100810081


In [145]:
# Lemma CountVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_count2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Bigrama com Naive Bayes', score])
print('Lemma CountVectorizer Bigrama com Naive Bayes:', score)

Lemma CountVectorizer Bigrama com Naive Bayes: 0.6765456209948124


In [146]:
# Lemma CountVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_count3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Trigrama com Naive Bayes', score])
print('Lemma CountVectorizer Trigrama com Naive Bayes:', score)

Lemma CountVectorizer Trigrama com Naive Bayes: 0.6257240652975249


##### CountVectorizer + Stopwords

In [147]:
# Lemma CountVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_count1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Unigrama e stopwords com Naive Bayes', score])
print('Lemma CountVectorizer Unigrama e stopwords com Naive Bayes:', score)

Lemma CountVectorizer Unigrama e stopwords com Naive Bayes: 0.7217784050811573


In [148]:
# Lemma CountVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_count2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Bigrama e stopwords com Naive Bayes', score])
print('Lemma CountVectorizer Bigrama e stopwords com Naive Bayes:', score)

Lemma CountVectorizer Bigrama e stopwords com Naive Bayes: 0.63


In [149]:
# Lemma CountVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_count3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_count3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma CountVectorizer Trigrama e stopwords com Naive Bayes', score])
print('Lemma CountVectorizer Trigrama e stopwords com Naive Bayes:', score)

Lemma CountVectorizer Trigrama e stopwords com Naive Bayes: 0.6577649835730917


##### TfidfVectorizer

In [150]:
# Lemma TfidfVectorizer Unigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_tfidf1Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf1Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Unigrama com Naive Bayes', score])
print('Lemma TfidfVectorizer Unigrama com Naive Bayes:', score)

Lemma TfidfVectorizer Unigrama com Naive Bayes: 0.7504672250826097


In [151]:
# Lemma TfidfVectorizer Bigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_tfidf2Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf2Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Bigrama com Naive Bayes', score])
print('Lemma TfidfVectorizer Bigrama com Naive Bayes:', score)

Lemma TfidfVectorizer Bigrama com Naive Bayes: 0.7332873376623377


In [152]:
# Lemma TfidfVectorizer Trigrama com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_tfidf3Vect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf3Vect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Trigrama com Naive Bayes', score])
print('Lemma TfidfVectorizer Trigrama com Naive Bayes:', score)

Lemma TfidfVectorizer Trigrama com Naive Bayes: 0.6198511487433934


##### TfidfVectorizer + Stopwords

In [153]:
# Lemma TfidfVectorizer Unigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_tfidf1StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf1StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Unigrama e stopwords com Naive Bayes', score])
print('Lemma TfidfVectorizer Unigrama e stopwords com Naive Bayes:', score)

Lemma TfidfVectorizer Unigrama e stopwords com Naive Bayes: 0.7329466162253047


In [154]:
# Lemma TfidfVectorizer Bigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_tfidf2StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf2StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Bigrama e stopwords com Naive Bayes', score])
print('Lemma TfidfVectorizer Bigrama e stopwords com Naive Bayes:', score)

Lemma TfidfVectorizer Bigrama e stopwords com Naive Bayes: 0.6275805626598465


In [155]:
# Lemma TfidfVectorizer Trigrama e stopwords com Naive Bayes
# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(lemma_tfidf3StopVect_train, df_train.sentimento)

# escoragem da classificação na amostra de teste
text_vect_test = lemma_tfidf3StopVect.transform(df_test.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, df_test.sentimento, average='weighted')
resultado.append(['Lemma TfidfVectorizer Trigrama e stopwords com Naive Bayes', score])
print('Lemma TfidfVectorizer Trigrama e stopwords com Naive Bayes:', score)

Lemma TfidfVectorizer Trigrama e stopwords com Naive Bayes: 0.6575895263765884


## Resultados

In [156]:
# Resumo de resultados
df_res = pd.DataFrame(np.unique(np.array(resultado), axis=0), columns=['Técnica', 'F1 Score'])
df_res.sort_values(by='F1 Score', ascending=False)


,Técnica,F1 Score
72,Texto CountVectorizer Bigrama com Naive Bayes,0.8001600640256101
92,Texto TfidfVectorizer Unigrama com Naive Bayes,0.799395220164451
80,Texto CountVectorizer Unigrama com Naive Bayes,0.7951076492210748
84,Texto TfidfVectorizer Bigrama com Naive Bayes,0.7910161186331399
82,Texto CountVectorizer Unigrama e stopwords com...,0.7900630063006301
...,...,...
1,Lemma CountVectorizer Bigrama com arvore de de...,0.595648452929559
50,Snowball CountVectorizer Bigrama e stopwords c...,0.586125
89,Texto TfidfVectorizer Trigrama com arvore de d...,0.5798850574712644
52,Snowball CountVectorizer Trigrama com Naive Bayes,0.5701291162045841


In [163]:
print('Modelo com maior F1 score:', df_res.iloc[72, 0])

Modelo com maior F1 score: Texto CountVectorizer Bigrama com Naive Bayes


## Validação do professor

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos (funções, limpeza, etc.), criação dos objetos de vetorização e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

Para elaboração desta solução, optamos por testar em uma amostra de 1000 frases, todas as combinações que julgamos apropriadas utilizando: 
 - Stopwords:
    - sem remoção de Stopwords
    - com remoção de Stopwords
 - Stemming e Lemma:
    - Texto original
    - RSLP Stemmer
    - Snowball Stemmer
    - Lemmatizaçào (via SpaCy)
 - Vectorização:  
    - CountVectorizer (unigramas, bigramas ou trigramas)
    - TfidfVectorizer (unigramas, bigramas ou trigramas)
 - Modelo:
    - Árvore de Decisão
    - Naive Bayes

Escolhemos o modelo que melhor performou para ser a resposta final. 

**Favor executar todas as células apartir deste ponto**


In [158]:
# Import e downlaod de bibliotecas utilizadas a seguir
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

df_full = pd.read_csv('https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv', encoding='utf-8')

df_train, df_test = train_test_split(df_full.copy(), test_size = 0.2, random_state = 42)

# CountVectorizer com bigrama usando o texto original
text_count2Vect = CountVectorizer(ngram_range=(2,2))
text_count2Vect.fit(df_train.texto)
text_count2Vect_train = text_count2Vect.transform(df_train.texto)

# treinamento do modelo Naive Bayes
bayes = MultinomialNB()
bayes.fit(text_count2Vect_train, df_train.sentimento)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [159]:
dfTeste = df_test

In [160]:
# escoragem da classificação na amostra de teste
text_vect_test = text_count2Vect.transform(dfTeste.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, dfTeste.sentimento, average='weighted')
print('F1 Score de teste utilizando o texto original com CountVectorizer, Bigramas e Naive Bayes:', score)

F1 Score de teste utilizando o texto original com CountVectorizer, Bigramas e Naive Bayes: 0.8843036455724294


Aqui implemente a parte do **script de teste** do seu modelo e imprima a métrica F1 Score no final do script.

Obs.: Pense que o professor irá subistituir seu dataframe de teste ( **dfTeste** ) por outro e testar a performance do seu modelo com dados que não foram utilizados no desenvolvimento.

In [161]:
'''
# PARTE DO PROFESSOR  
import pandas as pd  
arquivo = 'url do professor'  
dfValidacao = pd.read_csv(arquivo, encoding='utf-8')  
dfTeste = dfValidacao  
'''

"\n# PARTE DO PROFESSOR  \nimport pandas as pd  \narquivo = 'url do professor'  \ndfValidacao = pd.read_csv(arquivo, encoding='utf-8')  \ndfTeste = dfValidacao  \n"

In [162]:
# escoragem da classificação na amostra de teste
text_vect_test = text_count2Vect.transform(dfTeste.texto)
y_prediction = bayes.predict(text_vect_test)

# mensuração do resultado pela acurácia
score = f1_score(y_prediction, dfTeste.sentimento, average='weighted')
print('F1 Score de validação utilizando o texto original com CountVectorizer, Bigramas e Naive Bayes:', score)

F1 Score de validação utilizando o texto original com CountVectorizer, Bigramas e Naive Bayes: 0.8843036455724294
